In [2]:
from mxnet import nd
from mxnet import gluon
from mxnet import autograd

In [3]:
def transform(data, label):
    return data.astype('float32')/255, label.astype('float32')
mnist_train = gluon.data.vision.FashionMNIST(train=True, transform=transform)
mnist_test = gluon.data.vision.FashionMNIST(train=False, transform=transform)

In [4]:
batch_size = 256
train_data = gluon.data.DataLoader(mnist_train, batch_size, shuffle=True)
test_data = gluon.data.DataLoader(mnist_test, batch_size, shuffle=False)

In [8]:
from mxnet import gluon

net = gluon.nn.Sequential()
with net.name_scope():
    net.add(gluon.nn.Flatten())
    net.add(gluon.nn.Dense(256, activation="relu"))
    net.add(gluon.nn.Dense(10))
net.initialize()

In [7]:
def accuracy(output, label):
    return nd.mean(output.argmax(axis=1) == label).asscalar()
b = nd.array([1, 2, 3]) == nd.array([1, 5, 6])
print b
nd.mean(b)

def evaluate_accuracy(data_iterator, net):
    acc = 0
    for data, label in data_iterator:
        output = net(data)
        acc += accuracy(output, label)
    return acc / len(data_iterator)


[ 1.  0.  0.]
<NDArray 3 @cpu(0)>


In [12]:
softmax_cross_entropy = gluon.loss.SoftmaxCrossEntropyLoss()
trainer = gluon.Trainer(net.collect_params(), 'sgd', {'learning_rate': 0.5})


for epoch in range(5):
    train_loss = 0.
    train_acc = 0.
    for data, label in train_data:
        with autograd.record():
            output = net(data)
            loss = softmax_cross_entropy(output, label)
        loss.backward()
        trainer.step(batch_size)

        train_loss += nd.mean(loss).asscalar()
        train_acc += accuracy(output, label)

    test_acc = evaluate_accuracy(test_data, net)
    print("Epoch %d. Loss: %f, Train acc %f, Test acc %f" % (
        epoch, train_loss/len(train_data), train_acc/len(train_data), test_acc))

Epoch 0. Loss: 0.287956, Train acc 0.893251, Test acc 0.893652
Epoch 1. Loss: 0.280440, Train acc 0.896448, Test acc 0.886133
Epoch 2. Loss: 0.275861, Train acc 0.898221, Test acc 0.891992
Epoch 3. Loss: 0.266189, Train acc 0.900947, Test acc 0.895703
Epoch 4. Loss: 0.262640, Train acc 0.901840, Test acc 0.895703


In [ ]:
learning_rate = 0.5
Epoch 0. Loss: 0.706147, Train acc 0.743789, Test acc 0.781445
Epoch 1. Loss: 0.474864, Train acc 0.826053, Test acc 0.852539
Epoch 2. Loss: 0.411230, Train acc 0.847612, Test acc 0.856934
Epoch 3. Loss: 0.379692, Train acc 0.859846, Test acc 0.868066
Epoch 4. Loss: 0.356780, Train acc 0.867919, Test acc 0.881836